In [1]:
%load_ext autoreload
%autoreload 2
from pcfg_parse_gen import Pcfg, PcfgGenerator, CkyParse
import nltk
import benepar
from nltk.tree import Tree
from nltk.corpus.reader import BracketParseCorpusReader
#from train_grammar import *
import re

def print_tree(tree_string):
    tree_string = tree_string.strip()
    tree = nltk.Tree.fromstring(tree_string)
    tree.pretty_print()

def draw_tree(tree_string):
    tree_string = tree_string.strip()
    tree = nltk.Tree.fromstring(tree_string)
    tree.draw()

In [2]:
benepar.download('benepar_en')
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package benepar_en to C:\nltk_data...
[nltk_data]   Package benepar_en is already up-to-date!
[nltk_data] Downloading package punkt to C:\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


True

In [3]:
bst_parser = benepar.Parser("benepar_en")

In [4]:
with open('allowed_words.txt') as fh:
    allowed_words = [line.strip() for line in fh]

In [5]:
# add more scienarios into this function
def correct_tag(inputStr):
    '''
    replace punctuation to avoid cycle, return the new tag
    inputStr -- string, string of input tag
    '''
    outputStr = inputStr.replace(',','Pause')
    outputStr = outputStr.replace('.', 'Punc')
    return outputStr

In [6]:
def find_rules(sents, parser, isS1, allowed_words, exclusion_dict=None,):
    '''
    find the rules from the trees parsed PennTree parser
    sents -- list of strings, a list of sentences
    parser -- a nltk parser, benepar in this case
    isS1 -- bool, specify if this is for S1 rules
    allowed_words -- list of strings, this is used to identify which are grammars and which are vocabs
    exclusion_dict -- dictionary, contains the rules we want to exclude in S2
                      It is not none when isS1 is False
    '''
    print(f'================{len(sents)} sentences in total================')
    num_finished = 0
    trees = [str(parser.parse(sent)) for sent in sents]
    count_dict = dict()
    isHeadFlag = True
    if isS1:
        division = 'S1'
    else:
        division = 'S2'
    for tree in trees:
        if (num_finished+1) % 100 == 0:
            print(f'================{num_finished+1} sentences finished================')
        test_tree = Tree.fromstring(tree)
        test_tree.chomsky_normal_form(horzMarkov=2)
        for subtree in test_tree.subtrees():
            if len(subtree.label().strip())<1:
                continue
            if isHeadFlag:
                headRule = (division, (subtree.label(), '<Unary>'))
                if headRule not in count_dict.keys():
                    count_dict[headRule] = 1
                    isHeadFlag = False
            parent = subtree.label()
            if len(subtree) == 2:
                try:
                    child_left = subtree[0].label()
                except:
                    child_left = subtree[0]
                try:
                    child_right = subtree[1].label()
                except:
                    child_right = subtree[1]
            elif len(subtree) == 1:
                try:
                    child_left = subtree[0].label()
                except:
                    child_left = subtree[0]
                child_right = '<Unary>'
            else:
                continue
            if len(child_left.strip())<1 or len(child_right.strip())<1:
                continue
            # if this should belong to vocab, skip it
            if child_left in allowed_words:
                continue
            this_rule = (parent, (child_left, child_right))
            if not isS1:
                if this_rule not in exclusion_dict.keys():
                    if this_rule not in count_dict.keys():
                        count_dict[this_rule] = 1
            else:
                count_dict[this_rule] = 1
            #print(parse_gram.rules[i][2])
        isHeadFlag = True
        num_finished += 1
    return count_dict

In [7]:
%%time
with open('example_sentences.txt') as fh:
    sents = [line.strip() for line in fh]
s1_dict = find_rules(sents, bst_parser, True, allowed_words)

================27 sentences in total================
Wall time: 27 s


In [8]:
%%time
with open('devset.txt') as fh:
    sents = [line.strip() for line in fh]
s2_dict = find_rules(sents, bst_parser, False, allowed_words, s1_dict)

================500 sentences in total================
================100 sentences finished================
================200 sentences finished================
================300 sentences finished================
================400 sentences finished================
================500 sentences finished================
Wall time: 1min 54s


In [9]:
def output_rules(OutputFileName, count_dict):
    '''
    store the count_dict locally with correct format
    OutputFileName -- string, the name of the output file
    count_dict -- dictionary, contains the updated count
    '''    
    f_output = open(OutputFileName, "w")
    for key, value in count_dict.items():
        if key[1][1] == '<Unary>':
            f_output.write(f'{value}\t{key[0]}\t{key[1][0]}\n')
        else:
            f_output.write(f'{value}\t{key[0]}\t{key[1][0]} {key[1][1]}\n')
    f_output.close()

In [10]:
output_rules('S1_raw.gr', s1_dict)
output_rules('S2_raw.gr', s2_dict)

In [11]:
#parse_gram = Pcfg(["S1_yabing.gr","S2_yabing.gr","Vocab_yabing.gr"])
parse_gram = Pcfg(["S1_raw.gr","S2_raw.gr","tagged_allowed_words.txt"])

#reading grammar file: S1_raw.gr
#reading grammar file: S2_raw.gr
#Ignored cycle `` -> ``
#Ignored cycle NP -> NP
#reading grammar file: tagged_allowed_words.txt


In [12]:
def update_weight(sents, parse_gram, count_dict):
    '''
    update the weight recorded in count_dict, return the updated count_dict
    sents -- list of strings, a list of sentences
    parse_gram -- Pcfg instance
    count_dict -- dictionary, contains the old count
    '''
    parser = CkyParse(parse_gram, beamsize=0.0001)
    ce, trees = parser.parse_sentences(sents)
    for tree in trees:
        test_tree = Tree.fromstring(tree)
        for subtree in test_tree.subtrees():
            parent = subtree.label()
            if len(subtree) == 2:
                try:
                    child_left = subtree[0].label()
                except:
                    child_left = subtree[0]
                try:
                    child_right = subtree[1].label()
                except:
                    child_right = subtree[1]
            elif len(subtree) == 1:
                try:
                    child_left = subtree[0].label()
                except:
                    child_left = subtree[0]
                child_right = '<Unary>'
            else:
                continue
            for i in parse_gram.rhs[(child_left, child_right)]:
                if parse_gram.rules[i][0] == parent:
                    this_rule = (parse_gram.rules[i][0], parse_gram.rules[i][1])
                    if this_rule in count_dict.keys():
                        count_dict[this_rule] += 1
                    else:
                        count_dict[this_rule] = 2
                    #print(parse_gram.rules[i][2])
    # if the original rules were not utilized, add them to count_dict
    for key, value in parse_gram.rules.items():
        this_rule = (value[0], value[1])
        if this_rule not in count_dict.keys():
            count_dict[this_rule] = value[2]
    return count_dict

In [13]:
def rule2gr(s1OutputFileName, s2OutputFileName, vocabOutputFileName, count_dict, s1InputFileName, s2InputFileName):
    '''
    store the count_dict locally with correct format
    s1OutputFileName -- string, the name of the output file for S1
    s2OutputFileName -- string, the name of the output file for S2
    vocabOutputFileName -- string, the name of the output file for Vocab
    count_dict -- dictionary, contains the updated count
    s1InputFileName -- string, the filename of the original S1
    s2InputFileName -- string, the filename of the original S2
    '''
    s1_keys = list()
    s2_keys = list()
    f_s1 = open(s1InputFileName, "r")
    for line in f_s1:
        line = line.strip()
        # skip the comment line
        if line and (not line.startswith("#")):
            # split the line and extract count, pos_tag, and terminal
            line_arr = re.split('[\s]+',line)
            if len(line_arr) == 4:
                s1_key = (line_arr[1], (line_arr[2], line_arr[3]))
            else:
                s1_key = (line_arr[1], (line_arr[2], '<Unary>'))
            if s1_key not in s1_keys: 
                s1_keys.append(s1_key)
    f_s1.close()
    
    f_s2 = open(s2InputFileName, "r")
    for line in f_s2:
        line = line.strip()
        # skip the comment line
        if line and (not line.startswith("#")):
            # split the line and extract count, pos_tag, and terminal
            line_arr = re.split('[\s]+',line)
            if len(line_arr) == 4:
                s2_key = (line_arr[1], (line_arr[2], line_arr[3]))
            else:
                s2_key = (line_arr[1], (line_arr[2], '<Unary>'))
            if s2_key not in s2_keys: 
                s2_keys.append(s2_key)
    f_s2.close()
    
    f_s1_output = open(s1OutputFileName, "w")
    f_s2_output = open(s2OutputFileName, "w")
    f_vocab_output = open(vocabOutputFileName, "w")
    isVocabFlag = True
    for key, value in count_dict.items():
        if key in s1_keys:
            if key[1][1] == '<Unary>':
                f_s1_output.write(f'{value}\t{key[0]}\t{key[1][0]}\n')
            else:
                f_s1_output.write(f'{value}\t{key[0]}\t{key[1][0]} {key[1][1]}\n')
            isVocabFlag = False
        if key in s2_keys:
            if key[1][1] == '<Unary>':
                f_s2_output.write(f'{value}\t{key[0]}\t{key[1][0]}\n')
            else:
                f_s2_output.write(f'{value}\t{key[0]}\t{key[1][0]} {key[1][1]}\n')
            isVocabFlag = False
        if isVocabFlag:
            if key[1][1] == '<Unary>':
                f_vocab_output.write(f'{value}\t{key[0]}\t{key[1][0]}\n')
            else:
                f_vocab_output.write(f'{value}\t{key[0]}\t{key[1][0]} {key[1][1]}\n')
        isVocabFlag = True
    f_s1_output.close()
    f_s2_output.close()
    f_vocab_output.close()

In [14]:
with open('example_sentences.txt') as fh:
    sents = [line.strip() for line in fh]
with open('devset.txt') as fh:
    sents += [line.strip() for line in fh]

count_dict = dict()
count_dict = update_weight(sents, parse_gram, count_dict)
rule2gr('S1_updated.gr', 'S2_updated.gr', 'Vocab_updated.gr', count_dict, 'S1_raw.gr', 'S2_raw.gr')

#parsing: ['Arthur', 'is', 'the', 'king', '.']
#No parses found for: Arthur is the king .
#parsing: ['Arthur', 'rides', 'the', 'horse', 'near', 'the', 'castle', '.']
#No parses found for: Arthur rides the horse near the castle .
#parsing: ['riding', 'to', 'Camelot', 'is', 'hard', '.']
#No parses found for: riding to Camelot is hard .
#parsing: ['do', 'coconuts', 'speak', '?']
#No parses found for: do coconuts speak ?
#parsing: ['what', 'does', 'Arthur', 'ride', '?']
#No parses found for: what does Arthur ride ?
#parsing: ['who', 'does', 'Arthur', 'suggest', 'she', 'carry', '?']
#No parses found for: who does Arthur suggest she carry ?
#parsing: ['why', 'does', 'England', 'have', 'a', 'king', '?']
#No parses found for: why does England have a king ?
#parsing: ['are', 'they', 'suggesting', 'Arthur', 'ride', 'to', 'Camelot', '?']
#No parses found for: are they suggesting Arthur ride to Camelot ?
#parsing: ['five', 'strangers', 'are', 'at', 'the', 'Round', 'Table', '.']
#No parses found fo

(TOP (Misc Arthur) (Misc is) (Misc the) (Misc king) (Misc .))
(TOP (Misc Arthur) (Misc rides) (Misc the) (Misc horse) (Misc near) (Misc the) (Misc castle) (Misc .))
(TOP (Misc riding) (Misc to) (Misc Camelot) (Misc is) (Misc hard) (Misc .))
(TOP (Misc do) (Misc coconuts) (Misc speak) (Misc ?))
(TOP (Misc what) (Misc does) (Misc Arthur) (Misc ride) (Misc ?))
(TOP (Misc who) (Misc does) (Misc Arthur) (Misc suggest) (Misc she) (Misc carry) (Misc ?))
(TOP (Misc why) (Misc does) (Misc England) (Misc have) (Misc a) (Misc king) (Misc ?))
(TOP (Misc are) (Misc they) (Misc suggesting) (Misc Arthur) (Misc ride) (Misc to) (Misc Camelot) (Misc ?))
(TOP (Misc five) (Misc strangers) (Misc are) (Misc at) (Misc the) (Misc Round) (Misc Table) (Misc .))
(TOP (Misc Guinevere) (Misc might) (Misc have) (Misc known) (Misc .))
(TOP (Misc Guinevere) (Misc should) (Misc be) (Misc riding) (Misc with) (Misc Patsy) (Misc .))
(TOP (Misc it) (Misc is) (Misc Sir) (Misc Lancelot) (Misc who) (Misc knows) (Misc Zoot) (

#No parses found for: the Holy Grail was covered by a yellow fruit .
#parsing: ['Zoot', 'might', 'have', 'been', 'carried', 'by', 'a', 'swallow', '.']
#No parses found for: Zoot might have been carried by a swallow .
#parsing: ['Arthur', 'rode', 'to', 'Camelot', 'and', 'drank', 'from', 'his', 'chalice', '.']
#No parses found for: Arthur rode to Camelot and drank from his chalice .
#parsing: ['they', 'migrate', 'precisely', 'because', 'they', 'know', 'they', 'will', 'grow', '.']
#No parses found for: they migrate precisely because they know they will grow .
#parsing: ['do', 'not', 'speak', '!']
#No parses found for: do not speak !
#parsing: ['Arthur', 'will', 'have', 'been', 'riding', 'for', 'eight', 'nights', '.']
#No parses found for: Arthur will have been riding for eight nights .
#parsing: ['Arthur', ',', 'sixty', 'inches', ',', 'is', 'a', 'tiny', 'king', '.']
#No parses found for: Arthur , sixty inches , is a tiny king .
#parsing: ['Arthur', 'knows', 'Patsy', ',', 'the', 'trusty', 

(TOP (Misc the) (Misc Holy) (Misc Grail) (Misc was) (Misc covered) (Misc by) (Misc a) (Misc yellow) (Misc fruit) (Misc .))
(TOP (Misc Zoot) (Misc might) (Misc have) (Misc been) (Misc carried) (Misc by) (Misc a) (Misc swallow) (Misc .))
(TOP (Misc Arthur) (Misc rode) (Misc to) (Misc Camelot) (Misc and) (Misc drank) (Misc from) (Misc his) (Misc chalice) (Misc .))
(TOP (Misc they) (Misc migrate) (Misc precisely) (Misc because) (Misc they) (Misc know) (Misc they) (Misc will) (Misc grow) (Misc .))
(TOP (Misc do) (Misc not) (Misc speak) (Misc !))
(TOP (Misc Arthur) (Misc will) (Misc have) (Misc been) (Misc riding) (Misc for) (Misc eight) (Misc nights) (Misc .))
(TOP (Misc Arthur) (Misc ,) (Misc sixty) (Misc inches) (Misc ,) (Misc is) (Misc a) (Misc tiny) (Misc king) (Misc .))
(TOP (Misc Arthur) (Misc knows) (Misc Patsy) (Misc ,) (Misc the) (Misc trusty) (Misc servant) (Misc .))
(TOP (Misc Arthur) (Misc and) (Misc Guinevere) (Misc migrate) (Misc frequently) (Misc .))
(TOP (Misc he) (Misc know

#No parses found for: We have ridden the length and breadth of the land in search of knights who will join me in my court of Camelot .
#parsing: ['I', 'must', 'speak', 'with', 'your', 'lord', 'and', 'master', '.']
#No parses found for: I must speak with your lord and master .
#parsing: ['What', ',', 'ridden', 'on', 'a', 'horse', '?']
#No parses found for: What , ridden on a horse ?
#parsing: ['Yes', '!']
#No parses found for: Yes !
#parsing: ["You're", 'using', 'coconuts', '!']
#using unseen part of speech for You're
#No parses found for: You're using coconuts !
#parsing: ['What', '?']
#No parses found for: What ?
#parsing: ["You've", 'got', 'two', 'empty', 'halves', 'of', 'coconut', 'and', 'you', "'re", 'bangin', "'em", 'together', '.']
#using unseen part of speech for You've
#No parses found for: You've got two empty halves of coconut and you 're bangin 'em together .
#parsing: ['So', '?']
#No parses found for: So ?
#parsing: ['We', 'have', 'ridden', 'since', 'the', 'snows', 'of', 'w

...

(TOP (Misc The) (Misc ferocity) (Misc of) (Misc the) (Misc French) (Misc taunting) (Misc took) (Misc him) (Misc completely) (Misc by) (Misc surprise) (Misc ,) (Misc and) (Misc Arthur) (Misc became) (Misc convinced) (Misc that) (Misc a) (Misc new) (Misc strategy) (Misc was) (Misc required) (Misc if) (Misc the) (Misc quest) (Misc for) (Misc the) (Misc Holy) (Misc Grail) (Misc were) (Misc to) (Misc be) (Misc brought) (Misc to) (Misc a) (Misc successful) (Misc conclusion) (Misc .))
(TOP (Misc Arthur) (Misc ,) (Misc having) (Misc consulted) (Misc his) (Misc closest) (Misc knights) (Misc ,) (Misc decided) (Misc that) (Misc they) (Misc should) (Misc separate) (Misc ,) (Misc and) (Misc search) (Misc for) (Misc the) (Misc Grail) (Misc individually) (Misc .))
(TOP (Misc Now) (Misc ,) (Misc this) (Misc is) (Misc what) (Misc they) (Misc did) (Misc --))
(TOP (Misc Greg) (Misc !))
(TOP (Misc The) (Misc Tale) (Misc of) (Misc Sir) (Misc Robin) (Misc ...) (Misc .))
(TOP (Misc So) (Misc each) (Misc of) 

#No parses found for: He was not in the least bit scared to be mashed into a pulp , Or to have his eyes gouged out , and his elbows broken .
#parsing: ['To', 'have', 'his', 'kneecaps', 'split', ',', 'and', 'his', 'body', 'burned', 'away', ',', 'And', 'his', 'limbs', 'all', 'hacked', 'and', 'mangled', ',', 'brave', 'Sir', 'Robin', '!']
#No parses found for: To have his kneecaps split , and his body burned away , And his limbs all hacked and mangled , brave Sir Robin !
#parsing: ['His', 'head', 'smashed', 'in', 'and', 'his', 'heart', 'cut', 'out', ',', 'And', 'his', 'liver', 'removed', 'and', 'his', 'bowels', 'unplugged', ',', 'And', 'his', 'nostrils', 'ripped', 'and', 'his', 'bottom', 'burned', 'off', ',', 'And', 'his', 'penis', '--']
#No parses found for: His head smashed in and his heart cut out , And his liver removed and his bowels unplugged , And his nostrils ripped and his bottom burned off , And his penis --
#parsing: ['That', "'s", '--', 'that', "'s", ',', 'uh', ',', 'that', "'s

(TOP (Misc He) (Misc was) (Misc not) (Misc in) (Misc the) (Misc least) (Misc bit) (Misc scared) (Misc to) (Misc be) (Misc mashed) (Misc into) (Misc a) (Misc pulp) (Misc ,) (Misc Or) (Misc to) (Misc have) (Misc his) (Misc eyes) (Misc gouged) (Misc out) (Misc ,) (Misc and) (Misc his) (Misc elbows) (Misc broken) (Misc .))
(TOP (Misc To) (Misc have) (Misc his) (Misc kneecaps) (Misc split) (Misc ,) (Misc and) (Misc his) (Misc body) (Misc burned) (Misc away) (Misc ,) (Misc And) (Misc his) (Misc limbs) (Misc all) (Misc hacked) (Misc and) (Misc mangled) (Misc ,) (Misc brave) (Misc Sir) (Misc Robin) (Misc !))
(TOP (Misc His) (Misc head) (Misc smashed) (Misc in) (Misc and) (Misc his) (Misc heart) (Misc cut) (Misc out) (Misc ,) (Misc And) (Misc his) (Misc liver) (Misc removed) (Misc and) (Misc his) (Misc bowels) (Misc unplugged) (Misc ,) (Misc And) (Misc his) (Misc nostrils) (Misc ripped) (Misc and) (Misc his) (Misc bottom) (Misc burned) (Misc off) (Misc ,) (Misc And) (Misc his) (Misc penis) (Mis

#No parses found for: Oh quick get the sword out I want to cut his head off !
#parsing: ['Oh', ',', 'cut', 'your', 'own', 'head', 'off', '!']
#No parses found for: Oh , cut your own head off !
#parsing: ['Yes', ',', 'do', 'us', 'all', 'a', 'favor', '!']
#No parses found for: Yes , do us all a favor !
#parsing: ['What', '?']
#No parses found for: What ?
#parsing: ['Yapping', 'on', 'all', 'the', 'time', '.']
#No parses found for: Yapping on all the time .
#parsing: ["You're", 'lucky', '.']
#using unseen part of speech for You're
#No parses found for: You're lucky .
#parsing: ["You're", 'not', 'next', 'to', 'him', '.']
#using unseen part of speech for You're
#No parses found for: You're not next to him .
#parsing: ['What', 'do', 'you', 'mean', '?']
#No parses found for: What do you mean ?
#parsing: ['You', 'snore', '.']
#No parses found for: You snore .
#parsing: ['Oh', 'I', 'do', "n't", '--', 'anyway', ',', 'you', "'ve", 'got', 'bad', 'breath', '.']
#No parses found for: Oh I do n't -- a

KeyError: ('Misc', 'Misc')

In [18]:
?find_rules